In [ ]:
import mysql.connector
import pandas as pd
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[6]").appName('reddit').getOrCreate()
combined_movies = spark.read.format("csv").options(inferschema='true',header='true',delimiter='\t').load('/home/ubuntu/cs179g_venv/src/combined.tsv')

# clean up NaNs and column names for post titles and their respective movies
combined_movies = combined_movies.filter(combined_movies.MovieTitle.contains("nan") == False)
combined_movies = combined_movies.withColumnRenamed("Title", "Post_Titles")
combined_movies = combined_movies.toPandas()
combined_movies.to("combined_movies_vw")

In [ ]:
# clean up imdb movies dataset
movies = pd.read_csv("/home/ubuntu/cs179g_venv/src/movies.csv", sep=",")
movies = spark.read.format("csv").options(inferschema='true',header='true').load('/home/ubuntu/cs179g_venv/src/movies.csv')
movies = movies.drop("movieId")
movies = movies.withColumnRenamed("title", "Movie_Titles")
movies.createOrReplaceTempView("movies_vw")

In [ ]:
# join 'combined_movies' and 'movies' and write to file
joined_df = spark.sql("""Select C.Post_Titles, M.Movie_Titles, M.genres from combined_movies_vw AS C inner join movies_vw AS M ON M.Movie_Titles = C.MovieTitle""")
joined_df.createOrReplaceTempView("joined_vw")
joined_df.write.csv("/home/ubuntu/cs179g_venv/src/movies_join.csv", sep='\t')
#joined_df.show(5)

In [ ]:
# might need to change the path for posts_csv because of spark execution
posts_csv = pd.read_csv("/home/ubuntu/cs179g_venv/src/movies_join.csv/part-00000-22c37347-9d31-4d1c-9e7c-4e2510bf79bf-c000.csv", sep="\t")
db_connection = mysql.connector.connect(user="mko022", password="password_daniel")

# create table containing posts, movie titles, and genres
db_cursor = db_connection.cursor()
db_cursor.execute("USE cs179g;")
db_cursor.execute("CREATE TABLE imdb(Post_Titles VARCHAR(510) NOT NULL, Movie_Titles VARCHAR(510) NOT NULL, genres VARCHAR(125), PRIMARY KEY(Post_Titles));")

# store posts titles and movie titles in string
posts_tuples = list(posts_csv.itertuples(index=False, name=None))
posts_string = str(posts_tuples).strip('[]')

# insert post titles and movie titles into table 'imdb'
db_cursor.execute("INSERT INTO imdb(Post_Titles, Movie_Titles, genres) VALUES " + posts_string + ";")
db_cursor.execute("FLUSH TABLES;")